# 🏠 Real Estate Price Analysis - EDA
This notebook is prepared to perform exploratory data analysis (EDA) on the real estate dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [ ]:
df = pd.read_csv("real_estate_data_utf8sig.csv")
df.head()

## 📋 Data Overview

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## 📊 Descriptive Statistics

In [ ]:
df.describe()

## 🔗 Correlation Matrix

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

## 📈 Price Distribution

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df['price'], bins=50, kde=True)
plt.title("Price Distribution")
plt.xlabel("Price (TL)")
plt.show()

## 🏘️ Average Price by Number of Rooms

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=df, x="number_of_rooms", y="price", estimator='mean')
plt.title("Average Price by Number of Rooms")
plt.ylabel("Price (TL)")
plt.xlabel("Number of Rooms")
plt.show()

## 📏 Relationship Between Area (sqm) and Price

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=df, x="area_sqm", y="price")
plt.title("Relationship Between Area and Price")
plt.xlabel("Area (sqm)")
plt.ylabel("Price (TL)")
plt.show()

## Bonus: Price Distribution by Number of Rooms (Boxplot)

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x="number_of_rooms", y="price")
plt.title("Price Distribution by Number of Rooms")
plt.xlabel("Number of Rooms")
plt.ylabel("Price (TL)")
plt.tight_layout()
plt.show()

## Bonus: Building Age vs. Price (Scatter + Trendline)

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(data=df, x="building_age", y="price", alpha=0.5)
sns.regplot(data=df, x="building_age", y="price", scatter=False, color="red")
plt.title("Building Age vs. Price")
plt.xlabel("Building Age (years)")
plt.ylabel("Price (TL)")
plt.tight_layout()
plt.show()

## Bonus: Average Price by City

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x="city", y="price", estimator=np.mean, ci=None)
plt.title("Average Price by City")
plt.xlabel("City")
plt.ylabel("Average Price (TL)")
plt.tight_layout()
plt.show()